# K-means clustering exercises

1. Look at the non-convex clusters below from the lecture notes.

a) Use k-means clustering with k = 2 to cluster it, visualise the results and evaluate the silohuette score of the clusters. Interpret the value.

b) Loop through values of k up to 10 and see how it affects the silohuette score. Plot your findings.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)
df = pd.DataFrame(X, columns=["x1", "x2"])
df.plot.scatter("x1", "x2", c=y, colormap="viridis", colorbar=False, title = "Non-convex clusters")

In [ ]:
# Solution 1a
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

model = KMeans(n_clusters=2)
model.fit(X)
clusterlabels = model.labels_
print(silhouette_score(X, clusterlabels))

# plot
df["clusterlabels"] = clusterlabels
df.plot.scatter("x1", "x2", c="clusterlabels", colormap="viridis", colorbar=False, title = "Non-convex clusters")

In [ ]:
# Solution 1b
sil_score = {}
for k in range(2, 11):
    model = KMeans(n_clusters=k)
    model.fit(X)
    clusterlabels = model.labels_
    sil_score[k] = silhouette_score(X, clusterlabels)
# plot

plt.plot(sil_score.keys(), sil_score.values())


2) Code k-means clustering in numpy using the below guidelines, taken from: https://www.deep-ml.com/problems/17 (highly recommend this website for learning ML!)

## K-Means Clustering Algorithm Implementation

### Initialization
Use the provided initial_centroids as your starting point. This step is already done for you in the input.

### Assignment Step
For each point in your dataset:
1. Calculate its distance to each centroid.
2. Assign the point to the cluster of the nearest centroid (use the Euclidean distance function).

### Update Step
For each cluster:
1. Calculate the mean of all points assigned to the cluster.
2. Update the centroid to this new mean position.
Hint: Be careful with potential empty clusters. Decide how you'll handle them (e.g., keep the previous centroid).

### Iteration
Repeat steps 2 and 3 until either:

1. The centroids no longer change significantly (this case does not need to be included in your solution), or
2. You reach the max_iterations limit.
Hint: You might want to keep track of the previous centroids to check for significant changes.

In [ ]:
def euclidean_distance(a, b):
    return np.sqrt(((a - b) ** 2).sum(axis=1))

def k_means_clustering(points: list[tuple[float, ...]], k: int, initial_centroids: list[tuple[float, ...]], max_iterations: int) -> list[tuple[float, ...]]:
	# Your code here
    final_centroids = []
    return final_centroids

In [ ]:
# Your function should past these tests:

assert k_means_clustering([(1, 2), (1, 4), (1, 0), (10, 2), (10, 4), (10, 0)], 2, [(1, 1), (10, 1)], 10) == [(1.0, 2.0), (10.0, 2.0)]

assert k_means_clustering([(0, 0, 0), (2, 2, 2), (1, 1, 1), (9, 10, 9), (10, 11, 10), (12, 11, 12)], 2, [(1, 1, 1), (10, 10, 10)], 10) == [(1.0, 1.0, 1.0), (10.3333, 10.6667, 10.3333)]

assert k_means_clustering([(1, 1), (2, 2), (3, 3), (4, 4)], 1, [(0,0)], 10) == [(2.5, 2.5)]

assert k_means_clustering([(0, 0), (1, 0), (0, 1), (1, 1), (5, 5), (6, 5), (5, 6), (6, 6),(0, 5), (1, 5), (0, 6), (1, 6), (5, 0), (6, 0), (5, 1), (6, 1)], 4, [(0, 0), (0, 5), (5, 0), (5, 5)], 10) == [(0.5, 0.5), (0.5, 5.5), (5.5, 0.5), (5.5, 5.5)]

In [ ]:
# Solution 2b

import numpy as np

def euclidean_distance(a, b):
    return np.sqrt(((a - b) ** 2).sum(axis=1))

def k_means_clustering(points, k, initial_centroids, max_iterations):
    points = np.array(points)
    centroids = np.array(initial_centroids)

    for iteration in range(max_iterations):
        # Assign points to the nearest centroid
        distances = np.array([euclidean_distance(points, centroid) for centroid in centroids])
        assignments = np.argmin(distances, axis=0)

        new_centroids = np.array([points[assignments == i].mean(axis=0) if len(points[assignments == i]) > 0 else centroids[i] for i in range(k)])

        # Check for convergence
        if np.all(centroids == new_centroids):
            break
        centroids = new_centroids
        centroids = np.round(centroids,4)
    return [tuple(centroid) for centroid in centroids]

# K-Nearest Neighbours Exercises

3. Look at the dataset from make_circles below.

a) Try and fit from values k = 1 -> 100 (use the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) function. Plot k against model score and choose which value of k is best, justify your decision.

b) Plot a [decision boundary](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.DecisionBoundaryDisplay.html) for the best score.

In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(n_samples=1000, noise = 0.05)

# plot it
plt.scatter(X[:, 0], X[:, 1], c=y, cmap="viridis")

In [ ]:
# Solution 3a
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split, GridSearchCV

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

parameters = {'n_neighbors': range(1, 101)}
knn = KNeighborsClassifier()
model = GridSearchCV(knn, parameters, cv=5)
model.fit(X_train, y_train)

cv_results = pd.DataFrame(model.cv_results_)
cv_results.plot.scatter("param_n_neighbors", "mean_test_score", yerr="std_test_score", figsize=(10,8))

# best model score and k value
bestscore = model.best_score_
optimal_k = model.best_params_['n_neighbors']

print(f"Best score: {bestscore}")
print(f"Optimal k: {optimal_k}")

In [ ]:
# Solution 3b
import seaborn as sns

DecisionBoundaryDisplay.from_estimator(model.best_estimator_, X_test, cmap="viridis")
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, palette="PRGn")

plt.show()

4. Given a list of points in n-dimensional space represented and a query point, implement a function to find the k nearest neighbors to the query point using Euclidean distance. Inspired by: https://www.deep-ml.com/problems/173



In [ ]:
import numpy as np

def k_nearest_neighbors(points, query_point, k):
    """
    Find k nearest neighbors to a query point

    Args:
        points: List of tuples representing points [(x1, y1), (x2, y2), ...]
        query_point: Tuple representing query point (x, y)
        k: Number of nearest neighbors to return

    Returns:
        List of k nearest neighbor points as tuples
    """
    if not points or k <= 0:
        return []

    if k > len(points):
        k = len(points)

    # Convert to numpy arrays for vectorized operations
    points_array = np.array(points)
    query_array = np.array(query_point)

    # Put the rest of your code here!

In [ ]:
# Your function should pass the following tests:

assert k_nearest_neighbors([(1, 2), (3, 4), (1, 1), (5, 6), (2, 3)], (2, 2), 3) == [(1, 2), (2, 3), (1, 1)]

assert k_nearest_neighbors([(0, 0), (1, 1), (2, 2), (3, 3)], (1.5, 1.5), 2) == [(1, 1), (2, 2)]

assert k_nearest_neighbors([(1, 1), (2, 2), (3, 3)], (0, 0), 1) == [(1, 1)]

In [ ]:
# Solution 4

import numpy as np

def k_nearest_neighbors(points, query_point, k):
    """
    Find k nearest neighbors to a query point

    Args:
        points: List of tuples representing points [(x1, y1), (x2, y2), ...]
        query_point: Tuple representing query point (x, y)
        k: Number of nearest neighbors to return

    Returns:
        List of k nearest neighbor points as tuples
    """
    if not points or k <= 0:
        return []

    if k > len(points):
        k = len(points)

    # Convert to numpy arrays for vectorized operations
    points_array = np.array(points)
    query_array = np.array(query_point)

    # Calculate Euclidean distances using broadcasting
    distances = np.sqrt(np.sum((points_array - query_array) ** 2, axis=1))

    # Get indices of k smallest distances
    # np.argsort is stable, so ties maintain original order
    k_nearest_indices = np.argsort(distances, kind='stable')[:k]

    # Return the k nearest points as tuples
    return [tuple(points_array[i]) for i in k_nearest_indices]